In [32]:
import plotly.express as px
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np 
%matplotlib inline

sns.set_style('darkgrid')
matplotlib.rcParams['font.size'] = 14
matplotlib.rcParams['figure.figsize'] = (10, 6)
matplotlib.rcParams['figure.facecolor'] = '#00000000'
import pandas as pd 
pd.set_option('display.max_columns', None)

In [3]:
dfDeliveries = pd.read_csv("../data/IPL_Ball_by_Ball_2008_2022.csv") 
dfMatches = pd.read_csv("../data/IPL_Matches_2008_2022.csv")

# Data Cleaning


removing matches where D/L is applied


In [4]:
dfMatches['method'].value_counts()

method
D/L    19
Name: count, dtype: int64

In [5]:
dfMatches['method'].unique()

array([nan, 'D/L'], dtype=object)

In [6]:
dfMatches = dfMatches[dfMatches['method'].isna()]

cleaning season column


In [7]:
dfMatches['Season'] = dfMatches['Season'].replace({'2020/21': '2020', '2009/10': '2010', '2007/08': '2008'})

cleaning team names


In [8]:
dfMatches['WinningTeam']=dfMatches['WinningTeam'].replace('Delhi Daredevils','Delhi Capitals')
dfMatches['WinningTeam']=dfMatches['WinningTeam'].replace('Kings XI Punjab','Punjab Kings') 
dfMatches['WinningTeam']=dfMatches['WinningTeam'].replace('Rising Pune Supergiants','Rising Pune Supergiant')

In [9]:
dfMatches['Team1']=dfMatches['Team1'].replace('Delhi Daredevils','Delhi Capitals')
dfMatches['Team1']=dfMatches['Team1'].replace('Kings XI Punjab','Punjab Kings') 
dfMatches['Team1']=dfMatches['Team1'].replace('Rising Pune Supergiants','Rising Pune Supergiant')

In [10]:
dfMatches['Team2']=dfMatches['Team2'].replace('Delhi Daredevils','Delhi Capitals')
dfMatches['Team2']=dfMatches['Team2'].replace('Kings XI Punjab','Punjab Kings') 
dfMatches['Team2']=dfMatches['Team2'].replace('Rising Pune Supergiants','Rising Pune Supergiant')

In [11]:
dfMatches['TossWinner']=dfMatches['TossWinner'].replace('Delhi Daredevils','Delhi Capitals')
dfMatches['TossWinner']=dfMatches['TossWinner'].replace('Kings XI Punjab','Punjab Kings') 
dfMatches['TossWinner']=dfMatches['TossWinner'].replace('Rising Pune Supergiants','Rising Pune Supergiant')

# merged data frame


In [12]:
dfMerged = dfDeliveries.merge(dfMatches, on='ID')

In [13]:
dfMerged['BattingTeam']=dfMerged['BattingTeam'].replace('Delhi Daredevils','Delhi Capitals')
dfMerged['BattingTeam']=dfMerged['BattingTeam'].replace('Kings XI Punjab','Punjab Kings') 
dfMerged['BattingTeam']=dfMerged['BattingTeam'].replace('Rising Pune Supergiants','Rising Pune Supergiant')

Bowling Team Column


In [14]:
dfMerged['BowlingTeam'] = np.where(dfMerged['BattingTeam']==dfMerged['Team1'],
                                       dfMerged['Team2'],
                                       dfMerged['Team1'])

# Batsman Stats


In [15]:
dfBatsmen = dfMerged.groupby('batter')\
            .agg({'ID':'nunique','batsman_run':'sum','isWicketDelivery':'sum'})\
            .reset_index()
dfBatsmen.columns = ['Batter','Innings','Runs','Dismissals']
dfBallsFaced = dfMerged[(dfMerged['extra_type']!='wides')].groupby('batter')['ID'].count().reset_index()
dfBallsFaced.columns = ['Batter','Balls Faced']
dfBatter0s = dfMerged[(dfMerged['batsman_run']==0)&(dfMerged['non_boundary']==0)]\
            .groupby('batter')['ID']\
            .count()\
            .reset_index()\
            .sort_values('ID')
dfBatter0s.columns = ['Batter','0s']
dfBatter1s = dfMerged[(dfMerged['batsman_run']==1)&(dfMerged['non_boundary']==0)]\
            .groupby('batter')['ID']\
            .count()\
            .reset_index()\
            .sort_values('ID')
dfBatter1s.columns = ['Batter','1s']
dfBatter2s = dfMerged[(dfMerged['batsman_run']==2)&(dfMerged['non_boundary']==0)]\
            .groupby('batter')['ID']\
            .count()\
            .reset_index()\
            .sort_values('ID')
dfBatter2s.columns = ['Batter','2s']
dfBatter4s = dfMerged[(dfMerged['batsman_run']==4)&(dfMerged['non_boundary']==0)]\
            .groupby('batter')['ID']\
            .count()\
            .reset_index()\
            .sort_values('ID')
dfBatter4s.columns = ['Batter','4s']
dfBatter6s = dfMerged[(dfMerged['batsman_run']==6)&(dfMerged['non_boundary']==0)]\
            .groupby('batter')['ID']\
            .count()\
            .reset_index()\
            .sort_values('ID')
dfBatter6s.columns = ['Batter','6s']
dfMatchWise = dfMerged.groupby(['batter','ID'])['batsman_run'].sum().reset_index()
dfHighScore = dfMatchWise.groupby(['batter'])['batsman_run'].max().reset_index()
dfHighScore.columns = ['Batter','High Score']

df25s = dfMatchWise[(dfMatchWise['batsman_run']>=25)&(dfMatchWise['batsman_run']<50)]\
        .groupby('batter')['batsman_run']\
        .count()\
        .reset_index()\
        .sort_values('batsman_run',ascending=False)
df25s.columns = ['Batter','25']

df50s = dfMatchWise[(dfMatchWise['batsman_run']>=50)&(dfMatchWise['batsman_run']<100)]\
        .groupby('batter')['batsman_run']\
        .count()\
        .reset_index()\
        .sort_values('batsman_run',ascending=False)
df50s.columns = ['Batter','50']

df75s = dfMatchWise[(dfMatchWise['batsman_run']>=75)&(dfMatchWise['batsman_run']<100)]\
        .groupby('batter')['batsman_run']\
        .count()\
        .reset_index()\
        .sort_values('batsman_run',ascending=False)
df75s.columns = ['Batter','75']

df100s = dfMatchWise[(dfMatchWise['batsman_run']>=100)]\
        .groupby('batter')['batsman_run']\
        .count()\
        .reset_index()\
        .sort_values('batsman_run',ascending=False)
df100s.columns = ['Batter','100']

dfBatsmen = dfBatsmen.merge(dfBallsFaced, on='Batter',how='outer').merge(dfBatter0s, on='Batter',how='outer').merge(dfBatter1s, on='Batter',how='outer').merge(dfBatter2s, on='Batter',how='outer').merge(dfBatter4s, on='Batter',how='outer')\
                     .merge(dfBatter6s, on='Batter',how='outer').merge(dfHighScore, on='Batter',how='outer')\
                     .merge(df25s, on='Batter',how='outer').merge(df50s, on='Batter',how='outer').merge(df75s, on='Batter',how='outer').merge(df100s, on='Batter',how='outer')
dfBatsmen['Strike Rate'] = (dfBatsmen['Runs']/dfBatsmen['Balls Faced'])*100
dfBatsmen['Batting Average'] = dfBatsmen['Runs']/dfBatsmen['Dismissals']
dfBatsmen.fillna(0, inplace=True)
dfBatsmen['notout'] = dfBatsmen['Innings'] - dfBatsmen['Dismissals']
dfBatsmen.sort_values(by='notout', ascending=False).head()

,Batter,Innings,Runs,Dismissals,Balls Faced,0s,1s,2s,4s,6s,High Score,25,50,75,100,Strike Rate,Batting Average,notout
339,MS Dhoni,201,4895,140,3616,1340.0,1490.0,328.0,337.0,226.0,84,65.0,23.0,3.0,0.0,135.370575,34.964286,61
423,RA Jadeja,156,2486,100,1954,732.0,855.0,180.0,182.0,88.0,62,32.0,2.0,0.0,0.0,127.226203,24.860000,56
27,AB de Villiers,167,5168,123,3395,1105.0,1415.0,268.0,412.0,253.0,133,37.0,41.0,11.0,3.0,152.223859,42.016260,44
136,DJ Bravo,110,1560,69,1204,498.0,447.0,111.0,120.0,66.0,70,13.0,5.0,0.0,0.0,129.568106,22.608696,41
597,YK Pathan,149,3143,108,2195,848.0,864.0,153.0,257.0,154.0,100,31.0,14.0,0.0,1.0,143.189066,29.101852,41


In [16]:
dfBatsmen.head(10)

,Batter,Innings,Runs,Dismissals,Balls Faced,0s,1s,2s,4s,6s,High Score,25,50,75,100,Strike Rate,Batting Average,notout
0,A Ashish Reddy,23,280,15,193,61.0,83.0,20.0,16.0,15.0,36,2.0,0.0,0.0,0.0,145.077720,18.666667,8
1,A Badoni,11,161,9,130,57.0,53.0,11.0,11.0,7.0,54,0.0,1.0,0.0,0.0,123.846154,17.888889,2
2,A Chandila,2,4,1,7,3.0,4.0,0.0,0.0,0.0,4,0.0,0.0,0.0,0.0,57.142857,4.000000,1
3,A Chopra,6,53,5,71,45.0,21.0,2.0,7.0,0.0,24,0.0,0.0,0.0,0.0,74.647887,10.600000,1
4,A Choudhary,3,25,2,20,4.0,13.0,1.0,1.0,1.0,15,0.0,0.0,0.0,0.0,125.000000,12.500000,1
5,A Dananjaya,1,4,0,5,2.0,2.0,1.0,0.0,0.0,4,0.0,0.0,0.0,0.0,80.000000,inf,1
6,A Flintoff,3,62,2,53,24.0,23.0,2.0,5.0,2.0,24,0.0,0.0,0.0,0.0,116.981132,31.000000,1
7,A Kumble,15,35,2,47,24.0,21.0,1.0,3.0,0.0,8,0.0,0.0,0.0,0.0,74.468085,17.500000,13
8,A Manohar,7,108,6,75,30.0,30.0,2.0,14.0,3.0,43,2.0,0.0,0.0,0.0,144.000000,18.000000,1
9,A Mishra,54,362,30,389,182.0,172.0,22.0,29.0,5.0,31,3.0,0.0,0.0,0.0,93.059126,12.066667,24


In [17]:
batsman_stats_df = dfBatsmen.rename(columns={'Batter': 'batter', 'Innings': 'matches_played','Runs':'runs_scored','Dismissals':'dismissals','Balls Faced':'balls_faced',
                                             '0s':'0s_scored','1s':'1s_scored','2s':'2s_scored','4s':'4s_scored','6s':'6s_scored','High Score':'high_score','25':'25_scored',
                                             '50':'50_scored','75':'75_scored','100':'100_scored','Strike Rate':'strike_rate','Batting Average':'batting_average'
                                             })

In [20]:
batsman_stats_df.head(15)

,batter,matches_played,runs_scored,dismissals,balls_faced,0s_scored,1s_scored,2s_scored,4s_scored,6s_scored,high_score,25_scored,50_scored,75_scored,100_scored,strike_rate,batting_average,notout,explosivity_rating
0,A Ashish Reddy,23,280,15,193,61.0,83.0,20.0,16.0,15.0,36,2.0,0.0,0.0,0.0,145.077720,18.666667,8,0.16
1,A Badoni,11,161,9,130,57.0,53.0,11.0,11.0,7.0,54,0.0,1.0,0.0,0.0,123.846154,17.888889,2,0.14
2,A Chandila,2,4,1,7,3.0,4.0,0.0,0.0,0.0,4,0.0,0.0,0.0,0.0,57.142857,4.000000,1,0.00
3,A Chopra,6,53,5,71,45.0,21.0,2.0,7.0,0.0,24,0.0,0.0,0.0,0.0,74.647887,10.600000,1,0.10
4,A Choudhary,3,25,2,20,4.0,13.0,1.0,1.0,1.0,15,0.0,0.0,0.0,0.0,125.000000,12.500000,1,0.10
5,A Dananjaya,1,4,0,5,2.0,2.0,1.0,0.0,0.0,4,0.0,0.0,0.0,0.0,80.000000,inf,1,0.00
6,A Flintoff,3,62,2,53,24.0,23.0,2.0,5.0,2.0,24,0.0,0.0,0.0,0.0,116.981132,31.000000,1,0.13
7,A Kumble,15,35,2,47,24.0,21.0,1.0,3.0,0.0,8,0.0,0.0,0.0,0.0,74.468085,17.500000,13,0.06
8,A Manohar,7,108,6,75,30.0,30.0,2.0,14.0,3.0,43,2.0,0.0,0.0,0.0,144.000000,18.000000,1,0.23
9,A Mishra,54,362,30,389,182.0,172.0,22.0,29.0,5.0,31,3.0,0.0,0.0,0.0,93.059126,12.066667,24,0.09


In [19]:
#explosivity rating = Total no. of boundaries hit / Total no. of balls faced
batsman_stats_df['explosivity_rating'] = round((batsman_stats_df['4s_scored']+batsman_stats_df['6s_scored'])/batsman_stats_df['balls_faced'],2)

# Bowling Stats


In [21]:
dfbowler = dfMerged.groupby('bowler')\
            .agg({'ID':'nunique','batsman_run':'sum','extras_run':'sum','isWicketDelivery':'sum'})\
            .reset_index()
dfbowler.columns = ['bowler','Innings','Runs','Extras','Dismissals']

dfBallsBowled = dfMerged[(dfMerged['extra_type']!='wides')].groupby('bowler')['ID'].count().reset_index()
dfBallsBowled.columns = ['bowler','Balls Bowled']

dfbowler4s = dfMerged[(dfMerged['batsman_run']==4)&(dfMerged['non_boundary']==0)]\
            .groupby('bowler')['ID']\
            .count()\
            .reset_index()\
            .sort_values('ID')
dfbowler4s.columns = ['bowler','4s']
dfbowler0s = dfMerged[(dfMerged['batsman_run']==0)&(dfMerged['non_boundary']==0)]\
            .groupby('bowler')['ID']\
            .count()\
            .reset_index()\
            .sort_values('ID')
dfbowler0s.columns = ['bowler','0s']
dfbowler1s = dfMerged[(dfMerged['batsman_run']==1)&(dfMerged['non_boundary']==0)]\
            .groupby('bowler')['ID']\
            .count()\
            .reset_index()\
            .sort_values('ID')
dfbowler1s.columns = ['bowler','1s']
dfbowler2s = dfMerged[(dfMerged['batsman_run']==2)&(dfMerged['non_boundary']==0)]\
            .groupby('bowler')['ID']\
            .count()\
            .reset_index()\
            .sort_values('ID')
dfbowler2s.columns = ['bowler','2s']

dfbowler6s = dfMerged[(dfMerged['batsman_run']==6)&(dfMerged['non_boundary']==0)]\
            .groupby('bowler')['ID']\
            .count()\
            .reset_index()\
            .sort_values('ID')
dfbowler6s.columns = ['bowler','6s']
dfMatchWise = dfMerged.groupby(['bowler','ID'])['batsman_run'].sum().reset_index()
dfHighScore = dfMatchWise.groupby(['bowler'])['batsman_run'].max().reset_index()
dfHighScore.columns = ['bowler','Highest Conceeded']

dfbowler = dfbowler.merge(dfBallsBowled, on='bowler',how='outer').merge(dfbowler4s, on='bowler',how='outer')\
                     .merge(dfbowler6s, on='bowler',how='outer').merge(dfbowler0s, on='bowler',how='outer').merge(dfbowler1s, on='bowler',how='outer').merge(dfbowler2s, on='bowler',how='outer').merge(dfHighScore, on='bowler',how='outer')
dfbowler['Strike Rate'] = (dfbowler['Balls Bowled']/dfbowler['Dismissals'])
dfbowler['Bowling Average'] = dfbowler['Runs']/dfbowler['Dismissals']
dfbowler['economy'] = (dfbowler['Runs']/dfbowler['Balls Bowled'])*6
dfbowler.fillna(0, inplace=True)
# Assuming dfbowler is your DataFrame and 'column1' and 'column2' are the columns you want to add
dfbowler['Runs Conceeded'] = dfbowler['Runs'] + dfbowler['Extras']

# If you want to rename the new column, you can use the rename method
new_column_name = 'Runs Conceded'
dfbowler.rename(columns={'Runs Conceeded': new_column_name}, inplace=True)
dfbowler

,bowler,Innings,Runs,Extras,Dismissals,Balls Bowled,4s,6s,0s,1s,2s,Highest Conceeded,Strike Rate,Bowling Average,economy,Runs Conceded
0,A Ashish Reddy,20,386,14,19,264,26.0,20.0,89.0,110.0,23.0,36,13.894737,20.315789,8.772727,400
1,A Badoni,2,11,1,2,12,0.0,0.0,4.0,5.0,3.0,6,6.000000,5.500000,5.500000,12
2,A Chandila,12,242,3,11,234,17.0,10.0,105.0,91.0,10.0,39,21.272727,22.000000,6.205128,245
3,A Choudhary,5,137,7,5,102,13.0,6.0,49.0,31.0,9.0,51,20.400000,27.400000,8.058824,144
4,A Dananjaya,1,46,1,0,24,2.0,4.0,7.0,10.0,2.0,46,inf,inf,11.500000,47
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
467,YK Pathan,79,1323,70,45,1105,96.0,55.0,442.0,491.0,53.0,44,24.555556,29.400000,7.183710,1393
468,YS Chahal,128,3465,175,168,2834,202.0,176.0,1120.0,1233.0,169.0,51,16.869048,20.625000,7.335921,3640
469,Yash Dayal,9,283,13,13,195,31.0,12.0,87.0,63.0,12.0,43,15.000000,21.769231,8.707692,296
470,Yuvraj Singh,72,1051,27,39,864,57.0,40.0,282.0,419.0,73.0,35,22.153846,26.948718,7.298611,1078


Need to find the number of single, double or more wickets in a match by bowler.


In [23]:
bowlgroup = dfMerged.groupby(['bowler'])

In [24]:
matches_played_by_bowler = bowlgroup['ID'].unique()

# Create a new dataframe with the bowler and match IDs
matches_played_df = pd.DataFrame({'Bowler': matches_played_by_bowler.index, 'Match IDs': matches_played_by_bowler.values})

matches_played_df

,Bowler,Match IDs
0,A Ashish Reddy,"[980915, 980907, 829773, 829731, 829727, 82971..."
1,A Badoni,"[1304109, 1304083]"
2,A Chandila,"[598052, 598047, 598036, 598032, 598026, 59802..."
3,A Choudhary,"[1082636, 1082633, 1082628, 1082621, 1082591]"
4,A Dananjaya,[1136569]
...,...,...
467,YK Pathan,"[1178394, 1136588, 1082627, 1082604, 1082593, ..."
468,YS Chahal,"[1312200, 1312199, 1312197, 1304114, 1304109, ..."
469,Yash Dayal,"[1312200, 1312197, 1304113, 1304108, 1304103, ..."
470,Yuvraj Singh,"[1136572, 1082630, 1082611, 981003, 980991, 98..."


In [25]:
# Assuming 'isWicketDelivery' column indicates if a wicket was taken
# Adjust the column name if necessary

# Create a list to store the results
wickets_details = []

# Iterate through each row in the matches_played_df dataframe
for index, row in matches_played_df.iterrows():
    bowler = row['Bowler']
    match_ids = row['Match IDs']
    
    # Iterate through each match ID
    for match_id in match_ids:
        # Filter the deliveries dataframe for the specific bowler and match ID
        bowler_match_deliveries = dfMerged[(dfMerged['bowler'] == bowler) & (dfMerged['ID'] == match_id)]
        
        # Count the number of wickets for this bowler in this match
        wickets_count = bowler_match_deliveries['isWicketDelivery'].sum()
        
        # Append the results to the list
        wickets_details.append({
            'Bowler': bowler,
            'Match ID': match_id,
            'Wickets': wickets_count
        })

# Create a DataFrame from the list
wickets_details_df = pd.DataFrame(wickets_details)

# Print the resulting dataframe
wickets_details_df


,Bowler,Match ID,Wickets
0,A Ashish Reddy,980915,1
1,A Ashish Reddy,980907,0
2,A Ashish Reddy,829773,2
3,A Ashish Reddy,829731,1
4,A Ashish Reddy,829727,0
...,...,...,...
11003,Z Khan,335998,1
11004,Z Khan,335996,3
11005,Z Khan,335992,1
11006,Z Khan,335985,4


In [26]:
# Assuming 'Wickets' column indicates the number of wickets taken in a match
# Adjust the column name if necessary

# Group by bowler and Wickets, and count the occurrences
wickets_count_per_bowler = wickets_details_df.groupby(['Bowler', 'Wickets']).size().reset_index(name='Count')

# Pivot the table for better visualization
wickets_count_per_bowler_pivot = wickets_count_per_bowler.pivot(index='Bowler', columns='Wickets', values='Count').fillna(0)

# Print the resulting DataFrame
wickets_count_per_bowler_pivot

Wickets,0,1,2,3,4,5,6
Bowler,,,,,,,
A Ashish Reddy,6.0,10.0,3.0,1.0,0.0,0.0,0.0
A Badoni,0.0,2.0,0.0,0.0,0.0,0.0,0.0
A Chandila,6.0,3.0,2.0,0.0,1.0,0.0,0.0
A Choudhary,1.0,3.0,1.0,0.0,0.0,0.0,0.0
A Dananjaya,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...
YK Pathan,49.0,18.0,9.0,3.0,0.0,0.0,0.0
YS Chahal,28.0,52.0,32.0,13.0,2.0,1.0,0.0
Yash Dayal,2.0,3.0,2.0,2.0,0.0,0.0,0.0


merging the two df


In [27]:
# Merge the DataFrames on the 'Bowler' and 'bowler' columns
bowler_merged_df = pd.merge(wickets_count_per_bowler_pivot, dfbowler, left_on='Bowler', right_on='bowler', how='inner')

# Drop the duplicate 'bowler' column (if needed)
# merged_df = merged_df.drop(columns='bowler')

# Print the resulting merged DataFrame
bowler_merged_df

,0,1,2,3,4,5,6,bowler,Innings,Runs,...,4s,6s,0s,1s,2s,Highest Conceeded,Strike Rate,Bowling Average,economy,Runs Conceded
0,6.0,10.0,3.0,1.0,0.0,0.0,0.0,A Ashish Reddy,20,386,...,26.0,20.0,89.0,110.0,23.0,36,13.894737,20.315789,8.772727,400
1,0.0,2.0,0.0,0.0,0.0,0.0,0.0,A Badoni,2,11,...,0.0,0.0,4.0,5.0,3.0,6,6.000000,5.500000,5.500000,12
2,6.0,3.0,2.0,0.0,1.0,0.0,0.0,A Chandila,12,242,...,17.0,10.0,105.0,91.0,10.0,39,21.272727,22.000000,6.205128,245
3,1.0,3.0,1.0,0.0,0.0,0.0,0.0,A Choudhary,5,137,...,13.0,6.0,49.0,31.0,9.0,51,20.400000,27.400000,8.058824,144
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,A Dananjaya,1,46,...,2.0,4.0,7.0,10.0,2.0,46,inf,inf,11.500000,47
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
467,49.0,18.0,9.0,3.0,0.0,0.0,0.0,YK Pathan,79,1323,...,96.0,55.0,442.0,491.0,53.0,44,24.555556,29.400000,7.183710,1393
468,28.0,52.0,32.0,13.0,2.0,1.0,0.0,YS Chahal,128,3465,...,202.0,176.0,1120.0,1233.0,169.0,51,16.869048,20.625000,7.335921,3640
469,2.0,3.0,2.0,2.0,0.0,0.0,0.0,Yash Dayal,9,283,...,31.0,12.0,87.0,63.0,12.0,43,15.000000,21.769231,8.707692,296
470,50.0,12.0,5.0,3.0,2.0,0.0,0.0,Yuvraj Singh,72,1051,...,57.0,40.0,282.0,419.0,73.0,35,22.153846,26.948718,7.298611,1078


In [33]:
bowler_merged_df.head(5)

,0,1,2,3,4,5,6,bowler,Innings,Runs,Extras,Dismissals,Balls Bowled,4s,6s,0s,1s,2s,Highest Conceeded,Strike Rate,Bowling Average,economy,Runs Conceded
0,6.0,10.0,3.0,1.0,0.0,0.0,0.0,A Ashish Reddy,20,386,14,19,264,26.0,20.0,89.0,110.0,23.0,36,13.894737,20.315789,8.772727,400
1,0.0,2.0,0.0,0.0,0.0,0.0,0.0,A Badoni,2,11,1,2,12,0.0,0.0,4.0,5.0,3.0,6,6.000000,5.500000,5.500000,12
2,6.0,3.0,2.0,0.0,1.0,0.0,0.0,A Chandila,12,242,3,11,234,17.0,10.0,105.0,91.0,10.0,39,21.272727,22.000000,6.205128,245
3,1.0,3.0,1.0,0.0,0.0,0.0,0.0,A Choudhary,5,137,7,5,102,13.0,6.0,49.0,31.0,9.0,51,20.400000,27.400000,8.058824,144
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,A Dananjaya,1,46,1,0,24,2.0,4.0,7.0,10.0,2.0,46,inf,inf,11.500000,47


In [30]:
list(bowler_merged_df.columns)

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 'bowler',
 'Innings',
 'Runs',
 'Extras',
 'Dismissals',
 'Balls Bowled',
 '4s',
 '6s',
 '0s',
 '1s',
 '2s',
 'Highest Conceeded',
 'Strike Rate',
 'Bowling Average',
 'economy',
 'Runs Conceded']

In [38]:
bowler_stats_df = bowler_merged_df.rename(columns={'Innings': 'matches_played','Runs':'runs_conceded','Dismissals':'wickets_taken','Balls Bowled':'balls_bowled',
                                             '0s':'0s_conceded','1s':'1s_conceded','2s':'2s_conceded','4s':'4s_conceded','6s':'6s_conceded','Highest Conceeded':'highest_conceded',
                                             'Strike Rate':'strike_rate','Bowling Average':'bowling_average','Runs Conceded':'total_runs_conceded','Extras':'extras_runs_conceded',
                                             0:'0_wickets_taken',1:'1_wickets_taken',2:'2_wickets_taken',3:'3_wickets_taken',4:'4_wickets_taken',5:'5_wickets_taken',
                                             6:'6_wickets_taken'
                                             })

In [39]:
bowler_stats_df.head(5)

,0_wickets_taken,1_wickets_taken,2_wickets_taken,3_wickets_taken,4_wickets_taken,5_wickets_taken,6_wickets_taken,bowler,matches_played,runs_conceded,extras_runs_conceded,wickets_taken,balls_bowled,4s_conceded,6s_conceded,0s_conceded,1s_conceded,2s_conceded,highest_conceded,strike_rate,bowling_average,economy,total_runs_conceded
0,6.0,10.0,3.0,1.0,0.0,0.0,0.0,A Ashish Reddy,20,386,14,19,264,26.0,20.0,89.0,110.0,23.0,36,13.894737,20.315789,8.772727,400
1,0.0,2.0,0.0,0.0,0.0,0.0,0.0,A Badoni,2,11,1,2,12,0.0,0.0,4.0,5.0,3.0,6,6.000000,5.500000,5.500000,12
2,6.0,3.0,2.0,0.0,1.0,0.0,0.0,A Chandila,12,242,3,11,234,17.0,10.0,105.0,91.0,10.0,39,21.272727,22.000000,6.205128,245
3,1.0,3.0,1.0,0.0,0.0,0.0,0.0,A Choudhary,5,137,7,5,102,13.0,6.0,49.0,31.0,9.0,51,20.400000,27.400000,8.058824,144
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,A Dananjaya,1,46,1,0,24,2.0,4.0,7.0,10.0,2.0,46,inf,inf,11.500000,47
